# Data Pulling

This notebook applies and tests the data pulling functions in the `src` directory

In [1]:
import sys
import os
from os.path import join
import yaml

project_dir = join(os.getcwd(), os.pardir)
sys.path.append(project_dir)

from src.data import pull_tweets

In [2]:
yaml_path = join(project_dir, 'configs.yaml')
configs = yaml.full_load(open(yaml_path, 'r'))['twitter_data_pull']
configs['save_path'] = join(project_dir, configs['save_path'])
configs['credentials_path'] = join(project_dir, configs['credentials_path'])
configs['yaml_key'] = configs['credentials_key']
del configs['credentials_key']

configs

{'credentials_path': '/Users/jfonseca/Desktop/research_projs/IWMI/solve-iwmi/notebooks/../twitter_creds.yaml',
 'save_path': '/Users/jfonseca/Desktop/research_projs/IWMI/solve-iwmi/notebooks/../data/raw/',
 'results_per_call': 500,
 'max_results': 3000,
 'from_date': '2020-05-14',
 'to_date': '2020-06-15',
 'query': '(cyclone amphan)',
 'verbose': True,
 'yaml_key': 'search_tweets_api'}

In [3]:
pull_tweets(**configs)

2020-07-22 15:50:10,739 - src.data._pull_data - INFO - Pulling raw Twitter data
2020-07-22 15:50:11,143 - searchtweets.credentials - WARNING - Grabbing bearer token from OAUTH
2020-07-22 15:50:11,145 - searchtweets.result_stream - INFO - using bearer token for authentication
2020-07-22 15:50:12,696 - searchtweets.result_stream - INFO - paging; total requests read so far: 1
2020-07-22 15:50:13,810 - searchtweets.result_stream - INFO - paging; total requests read so far: 2
2020-07-22 15:50:14,901 - searchtweets.result_stream - INFO - paging; total requests read so far: 3
2020-07-22 15:50:16,205 - searchtweets.result_stream - INFO - paging; total requests read so far: 4
2020-07-22 15:50:17,315 - searchtweets.result_stream - INFO - paging; total requests read so far: 5
2020-07-22 15:50:18,503 - searchtweets.result_stream - INFO - ending stream at 3000 tweets
2020-07-22 15:50:18,531 - src.data._pull_data - INFO - Data successfuly saved at "/Users/jfonseca/Desktop/research_projs/IWMI/solve-i